In [1]:
from pathlib import Path
import os
import re


FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day22.txt'

regex = r'(on|off).+?(-?\d+)..(-?\d+).+?(-?\d+)..(-?\d+).+?(-?\d+)..(-?\d+)'

with open(FOLDER / in_file) as f:
    data = re.findall(regex, f.read(), flags=re.MULTILINE)

In [2]:
class Cube():
    def __init__(self,x1, x2, y1, y2, z1, z2,on=True):
        self.turn_on = on
        self.x1 = min(x1, x2)
        self.x2 = max(x1, x2)
        self.y1 = min(y1, y2)
        self.y2 = max(y1, y2)
        self.z1 = min(z1, z2)
        self.z2 = max(z1, z2)
        
    @property
    def area(self):
        return abs(self.x1 - self.x2) * abs(self.y1 - self.y2) * abs(self.z1 - self.z2)

    def overlaps(self, other):
        return all([
           self.x2 > other.x1,
           self.x1 < other.x2,
           self.y2 > other.y1,
           self.y1 < other.y2,
           self.z2 > other.z1,
           self.z1 < other.z2
        ])
        
    def __or__(self, other):
        x_min = max(self.x1, other.x1) 
        x_max = min(self.x2, other.x2)
        y_min = max(self.y1, other.y1)
        y_max = min(self.y2, other.y2)
        z_min = max(self.z1, other.z1)
        z_max = min(self.z2, other.z2)
        if self.overlaps(other):
            return Cube(x_min, x_max, y_min, y_max, z_min, z_max)

    def __eq__(self, other):
        return all([
            self.x1 == other.x1,
            self.x2 == other.x2,
            self.y1 == other.y1,
            self.y2 == other.y2,
            self.z1 == other.z1,
            self.z2 == other.z2
        ])
    
    def __ge__(self, other):
        return all([
            self.x1 <= other.x1,
            self.x2 >= other.x2,
            self.y1 <= other.y1,
            self.y2 >= other.y2,
            self.z1 <= other.z1,
            self.z2 >= other.z2
        ])


    def __hash__(self):
        return hash((
            self.x1,
            self.x2,
            self.y1,
            self.y2,
            self.z1,
            self.z2
        ))
        
    def __repr__(self):
        return f"{self.__class__.__name__}({self.x1}, {self.x2}, {self.y1}, {self.y2}, {self.z1}, {self.z2})"


In [3]:
data_list = [[l[0] == 'on'] + [int(n) for n in l[1:]] for l in data]

cubes = []

for (on, x1, x2, y1, y2, z1, z2) in  data_list:
    cubes.append(Cube(x1, x2+1, y1, y2+1, z1, z2+1, on))

len(cubes)

420

In [4]:
def count_on(data):
    count = 0
    seen = set()
    for c in reversed(data):
        if c.turn_on:
            intersections = [c|cut for cut in seen if c|cut]
            count += c.area
            intersection_count = count_on(intersections)
            count -= intersection_count
        seen.add(c)
    return count


bounds = Cube(-50, 50, -50, 50, -50, 50)
limited_counts = count_on([c for c in cubes if bounds >= c])
print("Solution 1: ", limited_counts)

all_counts = count_on(cubes)
print("Solution 2: ", all_counts)




Solution 1:  590467
Solution 2:  1225064738333321
